# Time-varying loadings

**Exchange rates and macroeconomic fundamentals: Evidence of instabilities from time‐varying factor loadings**

Eric Hillebrand, Jakob Guldbæk Mikkelsen, Lars Spreng, Giovanni Urga

In [1]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import numpy as np
from latentfactors.core.tvp import LoglikNomean, KFNomean
from latentfactors import cfg
from scipy.optimize import minimize

k = 3
cfg.fldr = r"C:\Users\franc\My Drive\bin\latentfactors"
fx = pd.read_excel(fr"{cfg.fldr}/data/hmsu/fx.xlsx", index_col=0)
eco_us = pd.read_excel(fr"{cfg.fldr}/data/hmsu/usa.xlsx", index_col=0)

# Log-difference FX
dlog_fx = fx.apply(np.log).diff().reindex(eco_us.index).mul(-1)
t, n = dlog_fx.shape

## Factors estimation

In [2]:
# Standardize economic indicators
scaler = StandardScaler()
scaler.fit(eco_us)
X = scaler.transform(eco_us)

# Transform them into actors
np.random.seed(1234)
svd = TruncatedSVD(n_components=k)
svd.fit(X)
V = svd.components_  # Right singular vectors
S = svd.singular_values_

U = X @ V.T @ np.linalg.inv(np.diag(S))  # Left singular values
mF = np.multiply(U, np.sqrt(t))

## Estimate loadings

In [3]:
mBeta = {}

# For-loop
crncy = "AUD"

# OLS Estimates
r = dlog_fx[crncy].values.reshape(-1, 1)  # FX Series
b = np.linalg.inv(mF.T @ mF) @ mF.T @ r  # OLS Beta
r_hat = mF @ b  # OLS predicted values
e = r - r_hat
s_e = e.T @ e / t  # Asymptotic OLS variance
# TODO: Compute White VCV
# TODO: Compute HAC VCV
L_ols = -1/2 * t * np.log(s_e.item())  # Likelihood values for restricted model with all loadings constant
# mBeta[crncy] = b.flatten()

In [12]:
vP0 = np.concatenate([np.zeros((2 * k, 1)), np.log(s_e)])

opt = minimize(
    fun=LoglikNomean, 
    x0=vP0.flatten(), 
    args=(r, mF),
    method='SLSQP'
)

In [13]:
opt

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1192.5776914243527
       x: [ 7.699e-03 -5.020e+01  1.359e-02 -7.121e+01  5.380e-03
           -6.689e+01 -7.327e+00]
     nit: 5
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00 -4.883e-04]
    nfev: 43
    njev: 5

In [ ]:
from filterpy.kalman import KalmanFilter
